In [5]:
import pandas as pd
from konlpy.tag import Mecab
from functools import *
from collections import Counter
import numpy as np
from tqdm.auto import tqdm

In [6]:
def get_word(res):
    res_w = []
    for word in res:
        if len(word) < 2:
            continue
        else:
            res_w.append(word)
    return res_w

tokenizer = Mecab(dicpath=r"C:\Mecab\mecab-ko-dic")

def split_word_df(df_trg):
    res_list = []
    print(f'total : {len(df_trg)}')
    for content in tqdm(df_trg.contents_clean):
        res = tokenizer.nouns(content)
        res = get_word(res)
        res_list.append(res)
    df_trg['words'] = res_list
    return df_trg

def get_word_pct_dict(lst_word , df_src , trg_col):
    word_pct_dict = {}
    for word in tqdm(lst_word):
        word_pct_dict.update(  {word : len(df_src[df_src[trg_col].apply(lambda x: word in x)])/len(df_src)} )
        sorted_word_pct_dict = sorted(word_pct_dict.items(), key=lambda x:x[1], reverse=True)
    return sorted_word_pct_dict

def flattened_list(lst):
    flattened_lst = [item for sublist in lst for item in sublist]
    return flattened_lst

In [7]:
df = pd.read_parquet('../df_qesg28_final_time.pq')

# 전체 기준으로 단어 빈도 

In [8]:
df_temp = split_word_df(df)

total : 106164


  0%|          | 0/106164 [00:00<?, ?it/s]

In [9]:
# 저장
df_temp= df_temp.reset_index(drop= False)
df_temp.to_parquet('df_temp.pq')

# Load Data

In [10]:
df_temp = pd.read_parquet('df_temp.pq')

## 전체 코퍼스 만들기

In [11]:
# loop 로 전체 코퍼스 만들기
corpus = []
for val in tqdm(df_temp.words):
    corpus += list(val)

corpus= list(set(corpus))
print(len(corpus))

  0%|          | 0/106164 [00:00<?, ?it/s]

99688


In [12]:
## 오래 걸려서 안씀
'''
# 전체에서 90% 이상 나오는 단어는 제거 
df_trg = df_temp
# 단어 리스트 
word_set = []
for i , row in tqdm(df_trg.iterrows()):
    word_set += list(set(row.words))
word_set = list(set(word_set))
    

print(len(word_set))
count_mat = np.zeros_like(word_set , dtype = 'int32')
for i , word in tqdm(enumerate(word_set)):
    for k , row in df_trg.iterrows():
        if word in row.words:
            count_mat[i] += 1 
'''

"\n# 전체에서 90% 이상 나오는 단어는 제거 \ndf_trg = df_temp\n# 단어 리스트 \nword_set = []\nfor i , row in tqdm(df_trg.iterrows()):\n    word_set += list(set(row.words))\nword_set = list(set(word_set))\n    \n\nprint(len(word_set))\ncount_mat = np.zeros_like(word_set , dtype = 'int32')\nfor i , word in tqdm(enumerate(word_set)):\n    for k , row in df_trg.iterrows():\n        if word in row.words:\n            count_mat[i] += 1 \n"

# 특정 카테고리 기준으로 단어 빈도 

In [17]:
# 기간 지정 및 제외 기타 제거 
df = df_temp
df_sm = df.loc[df.contents_clean != '']\
    .loc[df.y != '제외']\
    .loc[df.y != '기타']\
    .loc[df.time < '2021-04-28']\
    .loc[df.time > '2020-01-01']

In [20]:
df_sm.y.value_counts()

에너지및온실가스       7708
사회공헌           6326
인적자원관리         5741
소유구조           4898
ESG거버넌스        2867
제품/서비스         2208
공급망관리          2044
사업장안전및보건       1704
내부통제/투명성       1492
소비자보호          1449
이사회구성과운영       1377
지역사회영향         1373
자원순환           1232
공정거래           1173
계열회사와의거래        519
주주가치보호          463
이익배분            416
감사기구            375
인권              263
환경관리체계          202
대기오염            167
공시              118
친환경제품/서비스개발      79
수질오염             79
생물다양성            49
화학물질배출           34
기타오염             22
토양오염              6
Name: y, dtype: int64

## 원하는 카테고리 저장

In [15]:
df_trg = df.loc[df.y == '환경관리체계']
df_trg.to_excel('환경관리체계.xlsx') 
#df_trg = pd.read_excel('환경관리체계.xlsx', index_col= 0)

In [ ]:
df_trg = df.loc[ df.y == '환경관리체계']


# 토크나이저 
- 단어별 등장 빈도로 필터링 하려 했으나, 사용하지 않음. 

In [25]:
# 전체에서 90% 이상 나오는 단어는 제거 

# 단어 리스트 
word_set = []
for i , row in df_trg.iterrows():
    word_set += list(set(row.words))

word_set = list(set(word_set))
    
print(len(word_set))
count_mat = np.zeros_like(word_set , dtype = 'int32')
for i , word in tqdm(enumerate(word_set)):
    for k , row in df_trg.iterrows():
        if word in row.words:
            count_mat[i] += 1 
        

    

4550


0it [00:00, ?it/s]

In [26]:
doc_count = len(df_trg) # total doc count 
ratio_mat = count_mat / doc_count # 뉴스 1개 등장 비율/ 전체 문서 등장 비율

In [27]:
# 단어별 TF / IDF 데이터 프레임화
df_ratio = pd.DataFrame()
df_ratio['word']  = word_set
df_ratio['ratio'] = ratio_mat

In [37]:
# 단어중 최빈도 단어의 비율 * 80 의 비율값
df_ratio_top = df_ratio.loc[df_ratio.ratio >= 0.8* df_ratio.ratio.max() ]
print(0.8* df_ratio.ratio.max())

0.5320574162679427


In [29]:
# 단어중 최빈도 단어의 비율 * 80 이상의 비율을 가진 단어 제거 
df_ratio_cuttop = df_ratio.loc[df_ratio.ratio < 0.9* df_ratio.ratio.max() ]

In [24]:
df_ratio_cuttop.sort_values(by = 'ratio', ascending=False)

,word,ratio
20186,기업,0.583732
1039,기업,0.583732
2332,기업,0.583732
9724,기업,0.583732
17136,기업,0.583732
...,...,...
19613,급등세,0.004785
5679,당사자,0.004785
19615,비엠,0.004785
19616,바이,0.004785
